In [ ]:
import pandas as pd
import csv, sys, re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import tree, metrics
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier

df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE , dtype= str, header = None, names=["instance", "text", "id", "sentiment","is_sarcastic"])

In [ ]:
#df=shuffle(df)
df.head(10)

In [ ]:
df.sentiment.value_counts()

In [ ]:
df.id.value_counts()

In [ ]:
text_data = np.array([])
for text in df.text:
    text_data = np.append(text_data,text)
text_data[0]

In [ ]:
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
    punctuations = '''!"&'()*+,-./:;<=>?[\]^`{|}~'''
    no_punct = ""
    for char in sample:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct


def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = remove_punctuation(sample)
    return sample


def myTokenizer(sample):
    """Customized tokenizer"""
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2]
    return new_words

In [ ]:
myPreprocessor(df.text[5])

In [ ]:
df.text[5]

In [ ]:
count = CountVectorizer(preprocessor=myPreprocessor, lowercase = False, tokenizer=myTokenizer ,max_features = 200)
bag_of_words = count.fit_transform(text_data)
print(count.get_feature_names())
print(count.vocabulary_)

In [ ]:
X = bag_of_words.toarray()

In [ ]:
#target classes
Y = np.array([])
for text in df.sentiment:
    Y = np.append(Y,text)
Y

In [ ]:
X_train = X[:1500]
X_test = X[1500:]
y_train = Y[:1500]
y_test = Y[1500:]

In [ ]:
#clf = MultinomialNB()
#clf = BernoulliNB()
clf = tree.DecisionTreeClassifier(criterion='entropy',random_state=0)
clf2 = DummyClassifier(strategy= 'most_frequent')
model = clf.fit(X_train, y_train)
model2 = clf2.fit(X_train,y_train)

In [ ]:
predicted_y = model.predict(X_test)
predicted_y2 = model2.predict(X_test)

In [ ]:
print(accuracy_score(y_test, predicted_y))
print(accuracy_score(y_test, predicted_y2))

In [ ]:
print(y_test, predicted_y)
print(model.predict_proba(X_test))
print(accuracy_score(y_test, predicted_y))
print(precision_score(y_test, predicted_y, average='micro'))
print(recall_score(y_test, predicted_y, average='micro'))
print(f1_score(y_test, predicted_y, average='micro', labels = np.unique(predicted_y)))
print(f1_score(y_test, predicted_y, average='macro', labels = np.unique(predicted_y)))
print(classification_report(y_test, predicted_y,output_dict= False, labels = np.unique(predicted_y)))